In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [7]:
def load_data(dataset_path):
    classes = {'Acne': 0, 'Bags': 1, 'Redness': 2}  # Map class names to labels
    X, y = [], []
    for class_name, label in classes.items():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"Folder not found: {class_path}")
            continue
        for subfolder in os.listdir(class_path):  # Iterate through subfolders (0-10)
            subfolder_path = os.path.join(class_path, subfolder)
            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):  # Iterate through images
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, (128, 128))  # Resize images to 128x128
                        X.append(img)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {img_path}")
    print(f"Loaded {len(X)} images from {len(classes)} classes.")
    return np.array(X), np.array(y)

In [13]:
def enhance_image(img):
    # Convert to grayscale and apply histogram equalization
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    
    # Denoise the image
    denoised = cv2.fastNlMeansDenoising(equalized, h=10)
    
    # Convert back to a 3-channel image
    enhanced = cv2.cvtColor(denoised, cv2.COLOR_GRAY2BGR)
    return enhanced

In [14]:
def segment_skin(img):
    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Define skin color range
    lower_skin = np.array([0, 48, 80], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # Create a mask for skin region
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Apply the mask to the image
    segmented = cv2.bitwise_and(img, img, mask=mask)
    return segmented


In [28]:
from skimage.feature import local_binary_pattern, hog

def extract_features(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    features, hog_image = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    # Normalize HOG features (if needed)
    features = features.astype('float')
    features /= (features.sum() + 1e-6)  # Normalize

    # # Extract LBP features
    # lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    # Flatten and normalize LBP histogram
    # hist, _ = np.histogram(lbp, bins=np.arange(0, 10), range=(0, 9))
    # hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-6)  # Normalize
    
    # return hist
    return features


In [36]:
# Load data
X, y = load_data('Dataset')

# Normalize images
X = X / 255.0

Loaded 470 images from 3 classes.


In [40]:
# # Enhance images
# X_enhanced = np.array([enhance_image((img * 255).astype(np.uint8)) for img in X])

# # Segment skin regions
# X_segmented = np.array([segment_skin(img) for img in X_enhanced])

# # Extract features for traditional models
# X_features = np.array([extract_features(img) for img in X_segmented])

# Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)
X_flat = X.reshape(X.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train SVM model
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate SVM
svm_predictions = svm_model.predict(X_test)
print("SVM Classification Report:\n", classification_report(y_test, svm_predictions))
print("SVM Accuracy:", accuracy_score(y_test, svm_predictions))

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.68      0.67        31
           1       0.83      0.73      0.77        33
           2       0.64      0.70      0.67        30

    accuracy                           0.70        94
   macro avg       0.71      0.70      0.70        94
weighted avg       0.71      0.70      0.70        94

SVM Accuracy: 0.7021276595744681


In [57]:
def predict_image(input_image_path, model, image_size=(128, 128)):
    # Read the input image
    img = cv2.imread(input_image_path)

    # Convert the image to grayscale (if required)
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Resize the image to the same size as the training images
    resized = cv2.resize(img, image_size)

    # Flatten the image to match the input format for SVC (2D array)
    input_vector = resized.flatten().reshape(1, -1)

    # Predict the class of the input image
    prediction = model.predict(input_vector)
    
    return prediction

input_image_path = './Input/red girl 3.png'
prediction = predict_image(input_image_path, svm_model, image_size=(128,128))
if prediction == 0:
    print(f'Predicted class: Acne')
elif prediction == 1:
    print(f'Predicted class: Eyebags')
elif prediction == 2:
    print(f'Predicted class: Redness')
    
# print(svm_model.predict('./Input/black ppl.jpg'))
# print(svm_model.predict('./Input/eyebags.jpg'))
# print(svm_model.predict('./Input/red girl.jpg'))

Predicted class: Redness


In [19]:
# Step 3: Deep Learning Approach with CNN

# Step 6: Deep Learning Approach with CNN
# Split data for deep learning (use segmented images directly)
X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(X_segmented, y, test_size=0.2, random_state=42)

# One-hot encode labels for deep learning model
y_train_dl = to_categorical(y_train_dl, num_classes=3)
y_test_dl = to_categorical(y_test_dl, num_classes=3)

# Define CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(X_train, y_train_dl, epochs=10, validation_split=0.2, batch_size=32)

# Evaluate the model
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test_dl)
print("CNN Accuracy:", cnn_accuracy)

Epoch 1/10


c:\Users\Hansel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 9), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (None, 9)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 9), dtype=float32)
  • training=True
  • mask=None